# Загружаем библиотеки и датафрейм
---

In [ ]:
import pandas as pd
import numpy as np

In [5]:

data = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

In [6]:
df = data.copy()

In [7]:
df['Rating'] = df['event_type'].map({'show_vacancy': 0,
                                     'preview_click_vacancy': 0,
                                     'click_favorite': 0,
                                     'preview_click_favorite': 0,
                                     'click_response': 1,
                                     'preview_click_response': 1,
                                     'click_contacts': 1,
                                     'preview_click_contacts': 1,
                                     'click_phone': 1,
                                     'preview_click_phone': 1})

* 1 show_vacancy - просмотр вакансии
* 2 preview_click_vacancy - клик по карточке вакансии
* 3 click_response - отклик со страницы вакансии
* 4 preview_click_response - отклик с карточки вакансии
* 5 click_favorite - добавление вакансии в избранное  со страницы вакансии
* 6 preview_click_favorite - добавление вакансии в избраное с карточки вакансии
* 7 click_contacts - клик на контакты со страницы вакансии
* 8 preview_click_contacts - клик на контакты из карточки вакансии
* 9 click_phone - клик на номер телефона, указанный в вакансии
* 10 preview_click_phone - клик на номер телефона из карточки вакансии

Жирным выделены искомые ""откликнется"" и ""позвонит"""

In [10]:
df.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,Rating
51620,2022-08-01,1659301200,113824,71e26b4bc1ef4dbf8bc6ed74aaf83f75,24ca6fbdec8a4c2f8037639deefc7881,show_vacancy,0
200951,2022-08-01,1659301200,113824,71e26b4bc1ef4dbf8bc6ed74aaf83f75,24ca6fbdec8a4c2f8037639deefc7881,preview_click_vacancy,0
116157,2022-08-01,1659301201,119870,acff18561d8b4eada7938b750182527c,None,show_vacancy,0
24322,2022-08-01,1659301206,120270,8018bf19b86d4d43891cfcb03fe9486b,242cb3fc1fec48e4aec9a081983af922,show_vacancy,0
59721,2022-08-01,1659301206,120270,8018bf19b86d4d43891cfcb03fe9486b,242cb3fc1fec48e4aec9a081983af922,preview_click_vacancy,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 7 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   event_date       object
 1   event_timestamp  int64 
 2   vacancy_id_      int64 
 3   cookie_id        object
 4   user_id          object
 5   event_type       object
 6   Rating           int64 
dtypes: int64(3), object(4)
memory usage: 656.5+ MB


Начинаем преобразовывать характеристики

In [7]:
df['vacancy_id_num'] = df['vacancy_id_'].astype('int64')

In [9]:
df['cookie_id_num'] = df['cookie_id'].astype('category').cat.codes
df['user_id_num'] = df['user_id'].astype('category').cat.codes

In [19]:
df.head()

51620     1659301200
200951    1659301200
116157    1659301201
24322     1659301206
59721     1659301206
Name: event_timestamp, dtype: int64

In [9]:
df = df.sort_values('event_timestamp')

In [29]:
# можно записать что бы потом не кликать
df.to_parquet('df_parquet')

СОЗДАЕМ таблицы для обучения
---

In [10]:
# Только числовые значения 
df_cut = df[['cookie_id_num', 'vacancy_id_num', 'Rating']] # проверял пока только эту

In [14]:
df_cut.to_parquet('df_cut.paruet')

In [11]:
# все значения в изначальном варианте
df_str = df[['cookie_id', 'vacancy_id_', 'Rating']]

In [11]:
df_str.to_parquet('df_str.paruet')

In [12]:
# видел что время оч влияет и с ним обучаются - пока не тестил
df_time = df[['cookie_id_num', 'vacancy_id_num', 'Rating', 'event_timestamp']]

In [8]:
df_time.to_parquet('df_time.paruet')

In [58]:
#df_cut_time = df[['cookie_id_num', 'vacancy_id_num', 'Rating', 'event_timestamp']]

In [59]:
#df_str_time = df[['cookie_id', 'vacancy_id_', 'Rating', 'event_timestamp']]

Начинаем обучение С разбиением датасета
---

Загружаем разные датасеты
---

In [2]:
df_str = pd.read_parquet('df_cut_str.parquet')

In [17]:
from surprise import accuracy, Dataset
from surprise import KNNBaseline
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [12]:
df = df_str

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12292588 entries, 51620 to 10914320
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   cookie_id    object
 1   vacancy_id_  int64 
 2   Rating       int64 
dtypes: int64(2), object(1)
memory usage: 375.1+ MB


In [28]:
df.head()

,cookie_id,vacancy_id_,Rating
51620,71e26b4bc1ef4dbf8bc6ed74aaf83f75,113824,0
200951,71e26b4bc1ef4dbf8bc6ed74aaf83f75,113824,0
116157,acff18561d8b4eada7938b750182527c,119870,0
24322,8018bf19b86d4d43891cfcb03fe9486b,120270,0
59721,8018bf19b86d4d43891cfcb03fe9486b,120270,0


In [14]:
df_first = df.iloc [:500000]
df_last = df.iloc [500000:]

In [7]:
df_first.head()

,cookie_id,vacancy_id_,Rating
0,0000c4548c3944c08972bbdc1fa4eb85,137659,1
1,0000c4548c3944c08972bbdc1fa4eb85,153975,1
2,0000c4548c3944c08972bbdc1fa4eb85,174953,2
3,0000c4548c3944c08972bbdc1fa4eb85,176171,1
4,0000c4548c3944c08972bbdc1fa4eb85,182445,1


In [15]:
df_last.to_csv('dop_data.csv')

In [18]:
# Загружаем данные в формате Surprise
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df_first[['cookie_id', 'vacancy_id_', 'Rating']], reader)

In [19]:
# Разбиваем данные на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=.25)

In [20]:
print(testset[:5])

[('e3eeeab1e25147608a72a6622b9bc50e', 221088, 0.0), ('fec9c3acb4c5441c9de346c68f62673a', 149920, 0.0), ('e4b1496719a94af5879757ff273c6cba', 142619, 0.0), ('cd9234b8bebf443a94d5014cf77b5917', 120267, 0.0), ('fcd2b59d24fd4f9fbfbc7cc652e2d610', 150283, 0.0)]


In [1]:
sim_options = {'user_based': False } # Item-based подход

In [21]:
# Обучаем модель на первой порции данных
algo = KNNBaseline(k=3, verbose=False)
trainset_small = trainset
algo.fit(trainset_small)

TypeError: KNNBaseline.fit() got an unexpected keyword argument 'sim_options'

In [13]:

# Добавляем новые данные и дообучаем модель
batch_size = 100000
for i, batch in enumerate(pd.read_csv('dop_data.csv', chunksize=batch_size)):
    new_data = Dataset.load_from_df(batch[['cookie_id', 'vacancy_id_', 'Rating']], reader)
    trainset_new = new_data.build_full_trainset()
    algo.fit(trainset_new, sim_options=sim_options)

    # Оцениваем качество модели на тестовой выборке
    if i % 10 == 0:
        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions, verbose=False)
        mae = accuracy.mae(predictions, verbose=False)
        print(f'Batch {i}, RMSE = {rmse}')
        print(f'Batch {i}, MAE = {mae}')

Batch 0, RMSE = 0.5525836607316507
Batch 10, RMSE = 0.5251501426271539
Batch 20, RMSE = 0.5099254923919941
Batch 30, RMSE = 0.5035759786596014
Batch 40, RMSE = 0.5020458875508748
Batch 50, RMSE = 0.5140198987768515
Batch 60, RMSE = 0.5036916794135271
Batch 70, RMSE = 0.530570302486268
Batch 80, RMSE = 0.49982636601730035
Batch 90, RMSE = 0.526015417018195
Batch 100, RMSE = 0.49957914104869633


MemoryError: Unable to allocate 11.5 GiB for an array with shape (39338, 39338) and data type float64

### Функции предсказания
---

In [14]:
from collections import defaultdict

In [15]:
def get_top_n(predictions, n=5):
    """
    Возвращает топ-n рекомендаций для каждого пользователя
    """
    
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [iid for (iid, _) in user_ratings[:n]]

    return top_n

In [ ]:
# делаем предсказания на тестовом наборе
predictions = algo.test(testset)

# выводим метрики качества
accuracy.rmse(predictions)
accuracy.mae(predictions)

In [85]:
# Получаем топ-5 рекомендаций для каждого пользователя
top_n = get_top_n(predictions)

In [89]:

# Получаем топ-5 рекомендаций для заданного пользователя
user_id = 40752
recommended_items = top_n[user_id][:5]

print(recommended_items)

[251823, 144605, 250327, 155190, 155190]


In [ ]:
def precision_recall_at_k(predictions, k=10, threshold = 4):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [ ]:
precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))